In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
train_path="/content/drive/MyDrive/dataset/veg-dataset/train-set"
test_path="/content/drive/MyDrive/dataset/veg-dataset/test-set"

from keras.preprocessing.image import ImageDataGenerator
train_datagen=ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
test_datagen=ImageDataGenerator(rescale=1)

x_train=train_datagen.flow_from_directory(train_path,target_size=(128,128),batch_size=2,class_mode='categorical')
x_test=test_datagen.flow_from_directory(test_path,target_size=(128,128),batch_size=2,class_mode='categorical')

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten

model=Sequential()
model.add(Convolution2D(32,(3,3),input_shape=(128,128,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(units=300,kernel_initializer='uniform',activation='relu'))

model.add(Dense(units=150,kernel_initializer='uniform',activation='relu'))
model.add(Dense(units=75,kernel_initializer='uniform',activation='relu'))
model.add(Dense(units=9,kernel_initializer='uniform',activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer="adam",metrics=["accuracy"])


Found 9016 images belonging to 10 classes.
Found 3241 images belonging to 10 classes.


In [ ]:
model.fit(x_train,steps_per_epoch=len(x_train),validation_data=x_test,validation_steps=len(x_test),epochs=1)

In [4]:
model.save('fruit.h5')
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 63, 63, 32)       0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 127008)            0         
                                                                 
 dense_4 (Dense)             (None, 300)               38102700  
                                                                 
 dense_5 (Dense)             (None, 150)               45150     
                                                                 
 dense_6 (Dense)             (None, 75)                11325     
                                                      

In [5]:
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
import numpy as nps

model=load_model('fruit.h5')

img=image.load_img('/content/drive/MyDrive/dataset/fruit-dataset/test/Apple___healthy/02a0f44c-9cbe-4a2e-9710-296692635ec6___RS_HL 5789.JPG',grayscale=False,target_size=(128,128))

img

x=image.img_to_array(img)
x=nps.expand_dims(x,axis=0)

pred=(model.predict(x) > 0.5).astype("int32")

pred

1/1 [==============================] - 0s 129ms/step


array([[0, 0, 0, 0, 1, 0, 0, 0, 0]], dtype=int32)

In [ ]:
import requests
from tensorflow.keras.preprocessing import image

from tensorflow.keras.models import load_model
import numpy as np
import pandas as pd
import tensorflow as tf
from flask import Flask, request ,  render_template, redirect, url_for
import os
from werkzeug.utils import secure_filename
from tensorflow.python.keras.backend import set_session

app = Flask(__name__)
model = load_model("fruit.h5")
@app.route('/')
def home():
  return render_template('home.html')

@app.route('/prediction')
def prediction():
  return render_template('prediction.html')

@app.route('/prediction',methods=['POST'])
def predict():
  if request.method=='POST':
    f= request.files['images']
    basepath=os.path.dirname(__file__)
    file_path==os.path.join(
        basepath, 'uploads',secure_filename(f.filename))
    f.save(file_path)
    img=image.load_img(file_path, target_size=(128,128))
    x=image.img_to_array(img)
    x=np.expand_dims(x, axis=0)
    plant=request.form['plant']
    print(plant)
    if(plant=="fruit"):
      preds=model.predict_classess(x)
      print(preds)
      df=pd.read_excel('precautions-veg.xlsx')
      print (df.iloc[preds[0]]['cautions'])
    else:
      pred=model.predict_classes(x)
      df=pd.read_excel('precautions-fruits.xlsx')
      print(df.iloc[preds[0]]['caution'])
      return df.iloc[preds[0]]['caution']

if __name__=="__main__":
  app.run(debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
